Modelo como proposto por Sambasivan & Yahya (2005)

In [ ]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

# Entrada de dados

In [ ]:
Produtos = [1]
Plantas = [1]
Periodos = [1]

In [ ]:
# demanda =
# capacidade_planta =
# tempo_producao = 
# tempo_setup =
# custo_producao =
# custo_setup =
# custo_transporte =
# custo_estoque =
# limite_produto =

# Modelagem

In [ ]:
model = pyo.ConcreteModel()

## Conjuntos

In [ ]:
# Produtos
model.I = pyo.Set(initialize=Produtos)  # Índice i relativo aos produtos
# Plantas
model.J = pyo.Set(initialize=Plantas)  # Índice j relativo às plantas
# Períodos
model.T = pyo.Set(initialize=Periodos)  # Índice t relativo aos periodos

I, J, T = model.I, model.J, model.T

## Parâmetros

In [ ]:
# TENHO QUE TERMINAR ESSA BUDEGA E VER COMO APLICAR CORRETAMENTE NA DOCUMENTAÇÂO
# Função para inicialização do parâmetro que limita a produção do produto i na planta j no período t
def limite_produto_planta(model, i, j, t):
    return min((model.cap[j, t] - model.f[i, j]) / model.b[j, t], sum(sum(model.d[i, j, r] for r in T) for j in J))

In [ ]:
# Demanda
model.d = pyo.Param(I * J * T, within=pyo.NonNegativeReals)
# Capacidade
model.cap = pyo.Param(J * T, within=pyo.NonNegativeReals)
# Tempo de produção
model.b = pyo.Param(I * J, within=pyo.NonNegativeReals)
# Tempo de setup
model.f = pyo.Param(I * J, within=pyo.NonNegativeReals)
# Custo de produção
model.c = pyo.Param(I * J, within=pyo.NonNegativeReals)
# Custo de setup
model.s = pyo.Param(I * J, within=pyo.NonNegativeReals)
# Custo de transporte
model.r = pyo.Param(J * J, within=pyo.NonNegativeReals)
# Custo de estoque
model.h = pyo.Param(I * J, within=pyo.NonNegativeReals)
# Limite de produção de produto por planta
model.lim = pyo.Param(I * J * T, within=pyo.NonNegativeReals)  # , initialize=limite_produto_planta (terminar de montar)

d, cap, b, f, c, s, r, h, lim = model.d, model.cap, model.b, model.f, model.c, model.s, model.r, model.h, model.lim

## Variáveis de decisão

In [ ]:
# Quantidade produzida
model.X = pyo.Var(I * J * T, within=pyo.NonNegativeReals)
# Quantidade estocada
model.Q = pyo.Var(I * J * T, within=pyo.NonNegativeReals)
# Quantidade transportada
model.W = pyo.Var(I * J * J * T, within=pyo.NonNegativeReals)
# Variável de setup (binária)
model.Z = pyo.Var(I * J * T, within=pyo.Binary)

X, Q, W, Z = model.X, model.Q, model.W, model.Z

## Função objetivo

In [ ]:
expr_objetivo = sum(sum(sum(c[i, j] * X[i, j, t] + h[i, j] * Q[i, j, t] + s[i, j] * Z[i, j, t] + sum(r[j, k] * W[i, j, k, t] for k in J if k != j) for t in T) for j in J) for i in I) 
model.obj = pyo.Objective(sense=pyo.minimize,
                          expr=expr_objetivo)

objetivo = model.obj

## Restrições

In [ ]:
# Balanço de estoque
model.r_balanco = pyo.ConstraintList()
for i in I:
    for j in J:
        for t in T:
            if t == 1:
                restr = Q[i, j, t] == X[i, j, t] - sum(W[i, j, k, t] for k in J if k != j) + sum(W[i, l, j, t] for l in J if l != j) - d[i, j, t]
            else:
                restr = Q[i, j, t] == Q[i, j, t-1] + X[i, j, t] - sum(W[i, j, k, t] for k in J if k != j) + sum(W[i, l, j, t] for l in J if l != j) - d[i, j, t]
            model.r_balanco.add(expr=restr)

In [ ]:
# Restrição que obriga setup
model.r_setup = pyo.ConstraintList()
for i in I:
    for j in J:
        for t in T:
            restr = X[i, j, t] <= lim[i, j, t] * Z[i, j, t]
            model.r_setup.add(expr=restr)

In [ ]:
# Restrição de capacidade
model.r_capacidade = pyo.ConstraintList()
for j in J:
    for t in T:
        restr = sum(b[i, j] * X[i, j, t] + f[i, j] * Z[i, j, t] for i in I) <= cap[j, t]
        model.r_capacidade.add(expr=restr)

# Solver

In [ ]:
solver = SolverFactory("gurobi")
resultado = solver.solve(model, tee=False)

# Resultados (método exato)

# Resultados (heurística)